# Spotify Exploration

As of now (8/17/2023) I have only done this in R. So before we create the functions, we will recreate the functionality of my R script. 

1. Install Necessary Packages
2. Set system environments appropriately for Spotify ID's
3. Experiment with the Spotipy package
a. Get songs from an artist
b. Using track_id, get the song info (length, tempo, danceability, energy)
4. How many songs do I have saved?
5. Read in names of all songs
6. Get relevant data from all songs
7. Manage the dataframe
8. Create playlist groupings by tempo
9. Create playlist groupings by danceability
10. Create playlist groupings by energy
11. Write Playlist
12. Work with functions

# 1. Import Packages

In [3]:
import config
import pandas as pd
import numpy as np
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import math
import time
import spotipy.util as util

# 2. Authorization

Here we'll keep an example code from Spotipy for client authorizaiton (search spotify database but not any user data)
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

playlists = sp.user_playlists('spotify')
while playlists:
    for i, playlist in enumerate(playlists['items']):
        print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

Set system environments

In [4]:
os.environ["SPOTIPY_CLIENT_ID"] = config.SPOTIPY_CLIENT_ID
os.environ["SPOTIPY_CLIENT_SECRET"] = config.SPOTIPY_CLIENT_SECRET
os.environ["SPOTIPY_REDIRECT_URI"] = config.SPOTIPY_REDIRECT_URI

Set scope & do a data pull

* Note that "user-library-read" is the only necessary scope to read saved tracks

In [36]:
scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"
username = '1218158724'
token = util.prompt_for_user_token(username,scope,client_id=config.SPOTIPY_CLIENT_ID
                           ,client_secret=config.SPOTIPY_CLIENT_SECRET,redirect_uri=config.SPOTIPY_REDIRECT_URI)
sp = spotipy.Spotify(auth=token)

# profile = sp.current_user()
results = sp.current_user_saved_tracks(limit = 20, offset = 0, market = None)

# Item is a specific "row". Or a single song
for idx, item in enumerate(results['items']):
    
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'],'ID',track['id'])

0 Scotty Atl  –  Major ID 7Dzs4qqiiExHLgLnm8XxD8
1 Post Malone  –  Sign Me Up ID 4mahloUW5K6KeGijwnMGDr
2 Paul Russell  –  Ms. Poli Sci ID 3imYkN34og7muZbkSZWLUW
3 Kota the Friend  –  Barcelona (feat. Samm Henshaw) ID 1xNAaHsB48ycHgncrQAWgw
4 The Backseat Lovers  –  Kilby Girl ID 1170VohRSx6GwE6QDCHPPH
5 ROLE MODEL  –  blind ID 0NuWgxEp51CutD2pJoF4OM
6 The Backseat Lovers  –  Pool House ID 74f0dpqHhTeTJzs4pmZ1yq
7 Jack Harlow  –  Denver ID 5r30gHLxvhp60XMc5TIIMh
8 Landon Conrath  –  lowercase ID 7uy7ZDVB2E66cXEQJsC6Bz
9 Dominic Fike  –  Pasture Child ID 5APPlPgagEAXjGLIizqSDS
10 Dominic Fike  –  Sunburn ID 2Z90fzU2ZlMaUEgA2rhJxr
11 Dominic Fike  –  Bodies ID 59JXLBosh2OFLMCARkINnB
12 Dominic Fike  –  Sick ID 6OklSp2KgnW8RWNdQ7n8o1
13 Dominic Fike  –  Ant Pile ID 6OrCWnvAja4n5IYdzchJlH
14 Dominic Fike  –  How Much Is Weed? ID 44KWbTVZev3SWdv1t5UoYE
15 Young Thug  –  Hot (feat. Gunna) ID 5Z8HZM6iQMhhqyPcCGY5g9
16 Drake  –  First Person Shooter (feat. J. Cole) ID 7aqfrAY2p9BUSiupwk3svU
17

For each song that you get from current_user_saved_tracks, the variable will have two pieces ('added at', and 'track')

In [30]:
results['items'][0]['track']['id']

'7Dzs4qqiiExHLgLnm8XxD8'

In [31]:
print('you have', results['total'], 'saved songs')

you have 4471 saved songs


In [33]:
# item is a dictionary
print('track id: ', item['track']['id'])

print('track name: ', item['track']['name'])

print('track length in seconds: ',item['track']['duration_ms']/1000)

print('album name: ',item['track']['album']['name'])

print('first artist: ', item['track']['artists'][0]['name'])

print('number of artists: ', len(item['track']['artists']))


track id:  6ZprqcYFC6D9eIZaCoqrSn
track name:  Gently (feat. Bad Bunny)
track length in seconds:  133.694
album name:  For All The Dogs
first artist:  Drake
number of artists:  2


In [34]:
sp.audio_features('5Z8HZM6iQMhhqyPcCGY5g9')

[{'danceability': 0.855,
  'energy': 0.651,
  'key': 0,
  'loudness': -8.044,
  'mode': 0,
  'speechiness': 0.235,
  'acousticness': 0.063,
  'instrumentalness': 0,
  'liveness': 0.0772,
  'valence': 0.559,
  'tempo': 111.997,
  'type': 'audio_features',
  'id': '5Z8HZM6iQMhhqyPcCGY5g9',
  'uri': 'spotify:track:5Z8HZM6iQMhhqyPcCGY5g9',
  'track_href': 'https://api.spotify.com/v1/tracks/5Z8HZM6iQMhhqyPcCGY5g9',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5Z8HZM6iQMhhqyPcCGY5g9',
  'duration_ms': 193027,
  'time_signature': 4}]

# First loop
This loop is meant to go through all saved songs and get name, id, artist name, artist id, number of artists, and track length in seconds.

In [9]:
# Now we need to make a loop that will go through all saved songs
# Set scope
scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"
# Authorize
#sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
username = '1218158724'
token = util.prompt_for_user_token(username,scope,client_id=config.SPOTIPY_CLIENT_ID
                           ,client_secret=config.SPOTIPY_CLIENT_SECRET,redirect_uri=config.SPOTIPY_REDIRECT_URI)
sp = spotipy.Spotify(auth=token)
# Step 1: How many saved songs do we have?
# You can get this by doing a current_user_saved_tracks search
song = sp.current_user_saved_tracks(limit = 1, offset = 0, market = None)
num_songs = song['total']

# Initialize Variables
track_name = [] 
track_id = []
artist_name = []
artist_id = []
artist_num = []
track_len = []
# The limit for current_user_saved_tracks is 20, so we need to round up to the nearest multiple of 20
# in the math package the ceil function will round up
num_loops = math.ceil(song['total']/20)

# Now make a big loop that will go through it all
for i in range(0,num_loops+1):
    print('Loop Iteration',i+1)
    result = sp.current_user_saved_tracks(limit = 20, offset = (i*20), market = None)
    # sleep
    time.sleep(3)# 3 second sleep
    # Loop through saved tracks
    for item in result['items']:
        track = item['track']
        track_name.append(track['name'])
        track_id.append(track['id'])   
        artist_name.append(track['artists'][0]['name'])
        artist_id.append(track['artists'][0]['id'])
        artist_num.append(len(track['artists']))
        track_len.append(track['duration_ms']/1000)
# Convert to DF
Track_Name=pd.DataFrame(track_name,columns=['Track_Name'])
Track_ID=pd.DataFrame(track_id,columns=['Track_ID'])
Artist_Name=pd.DataFrame(artist_name,columns=['Artist_Name'])
Artist_ID=pd.DataFrame(artist_id,columns=['Artist_ID'])
Artist_Num=pd.DataFrame(artist_num,columns=['Artist_Num'])
Track_Len=pd.DataFrame(track_len,columns=['Track_Len'])
# Combine
df = pd.concat([Track_Name,Track_ID,Artist_Name,Artist_ID,Artist_Num,Track_Len],axis =1)
df



SpotifyOauthError: error: invalid_client, error_description: Invalid client

In [23]:
# Now we can do a loop that gets key info for each track
# This version will be slow because it's one at a time.
# Set scope
scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"
# Authorize
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

#trackfeatures = []
#for tracks in track_id:
    #trackfeatures.append( sp.audio_features(tracks = tracks) )
    #time.sleep(3)
#track_features = pd.DataFrame(trackfeatures)

# Step 1: How many saved songs do we have?
# You can get this by doing a current_user_saved_tracks search
song = sp.current_user_saved_tracks(limit = 1, offset = 0, market = None)
num_songs = song['total']
num_loops = math.ceil(song['total']/20)

# Realistically, we can do up to 100 searches at a time from the spotipy package
scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"
# Authorize
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

trackfeatures = pd.DataFrame(columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'])

for i in range(0,num_loops-1):
#for i in range(0,30):
    firstnum = i*20
    secondnum = firstnum +20
    #print(firstnum,secondnum)
    audio = (sp.audio_features(tracks = track_id[firstnum:secondnum]))
    audio = pd.DataFrame(audio)
    audio = audio.reset_index(inplace=False)
    
    trackfeatures = pd.concat([trackfeatures,audio],axis=0,ignore_index=True)
    
    time.sleep(2.1)
    idxval = num_loops-1-i
    print(idxval,'Loops to go! :)')
    if idxval <1:
        print("\a")# alarm?
        print('We have liftoff!')
        print('Come back to your code!')


# Now Merge with df
trackfeatures =trackfeatures.rename(columns={'id':'Track_ID'})
data = pd.merge(df,trackfeatures,on=['Track_ID'])
data

222 Loops to go! :)
221 Loops to go! :)
220 Loops to go! :)
219 Loops to go! :)
218 Loops to go! :)
217 Loops to go! :)
216 Loops to go! :)
215 Loops to go! :)
214 Loops to go! :)
213 Loops to go! :)
212 Loops to go! :)
211 Loops to go! :)
210 Loops to go! :)
209 Loops to go! :)
208 Loops to go! :)
207 Loops to go! :)
206 Loops to go! :)
205 Loops to go! :)
204 Loops to go! :)
203 Loops to go! :)
202 Loops to go! :)
201 Loops to go! :)
200 Loops to go! :)
199 Loops to go! :)
198 Loops to go! :)
197 Loops to go! :)
196 Loops to go! :)
195 Loops to go! :)
194 Loops to go! :)
193 Loops to go! :)
192 Loops to go! :)
191 Loops to go! :)
190 Loops to go! :)
189 Loops to go! :)
188 Loops to go! :)
187 Loops to go! :)
186 Loops to go! :)
185 Loops to go! :)
184 Loops to go! :)
183 Loops to go! :)
182 Loops to go! :)
181 Loops to go! :)
180 Loops to go! :)
179 Loops to go! :)
178 Loops to go! :)
177 Loops to go! :)
176 Loops to go! :)
175 Loops to go! :)
174 Loops to go! :)
173 Loops to go! :)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,index
0,0.686,0.897,11,-7.180,0,0.0386,0.010300,0.103000,0.153,0.355,129.975,audio_features,0FA4wrjDJvJTTU8AepZTup,spotify:track:0FA4wrjDJvJTTU8AepZTup,https://api.spotify.com/v1/tracks/0FA4wrjDJvJT...,https://api.spotify.com/v1/audio-analysis/0FA4...,163139,4,0.0
1,0.413,0.316,7,-7.794,1,0.0439,0.586000,0.000003,0.105,0.319,183.328,audio_features,4MTuL20LF3pWebeJbcNh7p,spotify:track:4MTuL20LF3pWebeJbcNh7p,https://api.spotify.com/v1/tracks/4MTuL20LF3pW...,https://api.spotify.com/v1/audio-analysis/4MTu...,183379,4,1.0
2,0.815,0.518,7,-6.594,0,0.0897,0.223000,0.000000,0.104,0.877,151.891,audio_features,0uI7yAKUf52Cn7y3sYyjiX,spotify:track:0uI7yAKUf52Cn7y3sYyjiX,https://api.spotify.com/v1/tracks/0uI7yAKUf52C...,https://api.spotify.com/v1/audio-analysis/0uI7...,177667,4,2.0
3,0.689,0.858,4,-2.868,1,0.1610,0.003830,0.000000,0.129,0.643,153.977,audio_features,7ACT6YaXbYvl7hRWEOOEHQ,spotify:track:7ACT6YaXbYvl7hRWEOOEHQ,https://api.spotify.com/v1/tracks/7ACT6YaXbYvl...,https://api.spotify.com/v1/audio-analysis/7ACT...,126465,4,3.0
4,0.445,0.797,11,-5.086,0,0.0434,0.000227,0.000000,0.363,0.267,135.483,audio_features,444vevlQjTnKioLLncteGv,spotify:track:444vevlQjTnKioLLncteGv,https://api.spotify.com/v1/tracks/444vevlQjTnK...,https://api.spotify.com/v1/audio-analysis/444v...,205288,4,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,0.628,0.601,1,-7.218,1,0.0336,0.745000,0.000000,0.310,0.744,114.988,audio_features,6g7SBkXVYCnnVVv8NE3lfV,spotify:track:6g7SBkXVYCnnVVv8NE3lfV,https://api.spotify.com/v1/tracks/6g7SBkXVYCnn...,https://api.spotify.com/v1/audio-analysis/6g7S...,192862,4,15.0
4396,0.508,0.763,1,-5.674,1,0.3030,0.062700,0.000000,0.347,0.275,161.590,audio_features,4zGvb8hxGLB2jEPRFiRRqw,spotify:track:4zGvb8hxGLB2jEPRFiRRqw,https://api.spotify.com/v1/tracks/4zGvb8hxGLB2...,https://api.spotify.com/v1/audio-analysis/4zGv...,192947,4,16.0
4397,0.430,0.955,9,-4.767,1,0.0424,0.000807,0.000000,0.044,0.366,172.975,audio_features,6HNruFQlzQx4ulL4ppRLYI,spotify:track:6HNruFQlzQx4ulL4ppRLYI,https://api.spotify.com/v1/tracks/6HNruFQlzQx4...,https://api.spotify.com/v1/audio-analysis/6HNr...,267360,4,17.0
4398,0.695,0.628,5,-8.627,1,0.1450,0.337000,0.219000,0.143,0.210,119.949,audio_features,4oRqSiVyGrwb8dnJBO6FNu,spotify:track:4oRqSiVyGrwb8dnJBO6FNu,https://api.spotify.com/v1/tracks/4oRqSiVyGrwb...,https://api.spotify.com/v1/audio-analysis/4oRq...,244653,4,18.0


In [32]:
def NumSavedSongs():
    # Set scope
    scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"
    # Authorize
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
    song = sp.current_user_saved_tracks(limit = 1, offset = 0, market = None)
    num_songs = song['total']
    return num_songs

Combine both loops

In [6]:
# Now we need to make a loop that will go through all saved songs
# Set scope
scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"
# Authorize
#sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
username = '1218158724'
token = util.prompt_for_user_token(username,scope,client_id=config.SPOTIPY_CLIENT_ID
                           ,client_secret=config.SPOTIPY_CLIENT_SECRET,redirect_uri=config.SPOTIPY_REDIRECT_URI)
sp = spotipy.Spotify(auth=token)
# Step 1: How many saved songs do we have?
# You can get this by doing a current_user_saved_tracks search
song = sp.current_user_saved_tracks(limit = 1, offset = 0, market = None)
num_songs = song['total']

# Initialize Variables
track_name = [] 
track_id = []
artist_name = []
artist_id = []
artist_num = []
track_len = []
trackfeatures = pd.DataFrame(columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'])
# The limit for current_user_saved_tracks is 20, so we need to round up to the nearest multiple of 20
# in the math package the ceil function will round up
num_loops = math.ceil(song['total']/20)

# Now make a big loop that will go through it all
for i in range(0,num_loops+1):
    print((num_loops+1-i),'Loops Left')
    result = sp.current_user_saved_tracks(limit = 20, offset = (i*20), market = None)
    # sleep
    time.sleep(8)# 3 second sleep
    # Loop through saved tracks
    for item in result['items']:
        track = item['track']
        track_name.append(track['name'])
        track_id.append(track['id'])  
        
        audio = (sp.audio_features(tracks = track['id']))
        audio = pd.DataFrame(audio)
        audio = audio.reset_index(inplace=False)
        trackfeatures = pd.concat([trackfeatures,audio],axis=0,ignore_index=True)
        
        artist_name.append(track['artists'][0]['name'])
        artist_id.append(track['artists'][0]['id'])
        artist_num.append(len(track['artists']))
        track_len.append(track['duration_ms']/1000)
# Convert to DF
Track_Name=pd.DataFrame(track_name,columns=['Track_Name'])
Track_ID=pd.DataFrame(track_id,columns=['Track_ID'])
Artist_Name=pd.DataFrame(artist_name,columns=['Artist_Name'])
Artist_ID=pd.DataFrame(artist_id,columns=['Artist_ID'])
Artist_Num=pd.DataFrame(artist_num,columns=['Artist_Num'])
Track_Len=pd.DataFrame(track_len,columns=['Track_Len'])
# Combine
df = pd.concat([Track_Name,Track_ID,Artist_Name,Artist_ID,Artist_Num,Track_Len],axis =1)
df

# Now Merge with df
trackfeatures =trackfeatures.rename(columns={'id':'Track_ID'})
data = pd.merge(df,trackfeatures,on=['Track_ID'])
data.to_csv('DantesSongsNov11.csv')

# took 28 min

225 Loops Left


Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=7Dzs4qqiiExHLgLnm8XxD8:
 Max Retries, reason: too many 429 error responses

In [ ]:
# Make a function to get all saved songs 
def SavedSongs():
    # Now we need to make a loop that will go through all saved songs
    # Set scope
    scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"
    # Authorize
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
    # Step 1: How many saved songs do we have?
    # You can get this by doing a current_user_saved_tracks search
    song = sp.current_user_saved_tracks(limit = 1, offset = 0, market = None)
    num_songs = song['total']

    # Initialize Variables
    track_name = [] 
    track_id = []
    artist_name = []
    artist_id = []
    artist_num = []
    track_len = []
    trackfeatures = pd.DataFrame(columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
           'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
           'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
           'time_signature'])
    # The limit for current_user_saved_tracks is 20, so we need to round up to the nearest multiple of 20
    # in the math package the ceil function will round up
    num_loops = math.ceil(song['total']/20)

    # Now make a big loop that will go through it all
    for i in range(0,num_loops+1):
        print((num_loops+1-i),'Loops Left')
        result = sp.current_user_saved_tracks(limit = 20, offset = (i*20), market = None)
        # sleep
        time.sleep(10)# 5 second sleep
        # Loop through saved tracks
        for item in result['items']:
            track = item['track']
            track_name.append(track['name'])
            track_id.append(track['id'])  
            audio = (sp.audio_features(tracks = track['id']))
            audio = pd.DataFrame(audio)
            audio = audio.reset_index(inplace=False)
            trackfeatures = pd.concat([trackfeatures,audio],axis=0,ignore_index=True)
         
            artist_name.append(track['artists'][0]['name'])
            artist_id.append(track['artists'][0]['id'])
            artist_num.append(len(track['artists']))
            track_len.append(track['duration_ms']/1000)
    # Convert to DF
    Track_Name=pd.DataFrame(track_name,columns=['Track_Name'])
    Track_ID=pd.DataFrame(track_id,columns=['Track_ID'])
    Artist_Name=pd.DataFrame(artist_name,columns=['Artist_Name'])
    Artist_ID=pd.DataFrame(artist_id,columns=['Artist_ID'])
    Artist_Num=pd.DataFrame(artist_num,columns=['Artist_Num'])
    Track_Len=pd.DataFrame(track_len,columns=['Track_Len'])
    # Combine
    df = pd.concat([Track_Name,Track_ID,Artist_Name,Artist_ID,Artist_Num,Track_Len],axis =1)

    # Now Merge with df
    trackfeatures =trackfeatures.rename(columns={'id':'Track_ID'})
    data = pd.merge(df,trackfeatures,on=['Track_ID'])
    return data

In [6]:
# Read data from pickle
data = pd.read_pickle("./DantesSongs.pkl")
data.head()

,Track_Name,Track_ID,Artist_Name,Artist_ID,Artist_Num,Track_Len,danceability,energy,key,loudness,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,index
0,Watch This - ARIZONATEARS Pluggnb Remix,0FA4wrjDJvJTTU8AepZTup,Lil Uzi Vert,4O15NlyKLIASxsJ0PrXPfz,3,163.139,0.686,0.897,11,-7.180,...,0.153,0.355,129.975,audio_features,spotify:track:0FA4wrjDJvJTTU8AepZTup,https://api.spotify.com/v1/tracks/0FA4wrjDJvJT...,https://api.spotify.com/v1/audio-analysis/0FA4...,163139,4,0.0
1,Don't Understand,4MTuL20LF3pWebeJbcNh7p,Post Malone,246dkjvS1zLTtiykXe5h60,1,183.379,0.413,0.316,7,-7.794,...,0.105,0.319,183.328,audio_features,spotify:track:4MTuL20LF3pWebeJbcNh7p,https://api.spotify.com/v1/tracks/4MTuL20LF3pW...,https://api.spotify.com/v1/audio-analysis/4MTu...,183379,4,0.0
2,3 Nights,0uI7yAKUf52Cn7y3sYyjiX,Dominic Fike,6USv9qhCn6zfxlBQIYJ9qs,1,177.666,0.815,0.518,7,-6.594,...,0.104,0.877,151.891,audio_features,spotify:track:0uI7yAKUf52Cn7y3sYyjiX,https://api.spotify.com/v1/tracks/0uI7yAKUf52C...,https://api.spotify.com/v1/audio-analysis/0uI7...,177667,4,0.0
3,Double Negative (Skeleton Milkshake),7ACT6YaXbYvl7hRWEOOEHQ,Dominic Fike,6USv9qhCn6zfxlBQIYJ9qs,1,126.465,0.689,0.858,4,-2.868,...,0.129,0.643,153.977,audio_features,spotify:track:7ACT6YaXbYvl7hRWEOOEHQ,https://api.spotify.com/v1/tracks/7ACT6YaXbYvl...,https://api.spotify.com/v1/audio-analysis/7ACT...,126465,4,0.0
4,Something Real,444vevlQjTnKioLLncteGv,Post Malone,246dkjvS1zLTtiykXe5h60,1,205.287,0.445,0.797,11,-5.086,...,0.363,0.267,135.483,audio_features,spotify:track:444vevlQjTnKioLLncteGv,https://api.spotify.com/v1/tracks/444vevlQjTnK...,https://api.spotify.com/v1/audio-analysis/444v...,205288,4,0.0


In [7]:
# experiment with filter
data[(data['tempo'] > 180) & (data['tempo'] < 190)]

,Track_Name,Track_ID,Artist_Name,Artist_ID,Artist_Num,Track_Len,danceability,energy,key,loudness,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,index
1,Don't Understand,4MTuL20LF3pWebeJbcNh7p,Post Malone,246dkjvS1zLTtiykXe5h60,1,183.379,0.413,0.316,7,-7.794,...,0.1050,0.319,183.328,audio_features,spotify:track:4MTuL20LF3pWebeJbcNh7p,https://api.spotify.com/v1/tracks/4MTuL20LF3pW...,https://api.spotify.com/v1/audio-analysis/4MTu...,183379,4,0.0
66,Murda Boyz - Remix,1Lr4Sgyn1lhUnHFrfOIvET,Hardo,3ohrdimoWGwbjGMOnTDoUJ,4,192.000,0.808,0.627,8,-6.855,...,0.7060,0.492,180.045,audio_features,spotify:track:1Lr4Sgyn1lhUnHFrfOIvET,https://api.spotify.com/v1/tracks/1Lr4Sgyn1lhU...,https://api.spotify.com/v1/audio-analysis/1Lr4...,192000,4,0.0
220,Mr. Big Shot,16zHGLd8US5ohzTcbJSetW,Anarbor,09J4zbHHwNNlN2tfgJIg21,1,205.506,0.414,0.923,6,-2.941,...,0.3570,0.602,185.002,audio_features,spotify:track:16zHGLd8US5ohzTcbJSetW,https://api.spotify.com/v1/tracks/16zHGLd8US5o...,https://api.spotify.com/v1/audio-analysis/16zH...,205507,4,0.0
224,My Way,26xfrX9phl1NNsz1H5TJJn,Logic,4xRYI6VqpkE3UwrDrAZL8L,1,156.530,0.608,0.667,8,-5.751,...,0.1190,0.150,184.021,audio_features,spotify:track:26xfrX9phl1NNsz1H5TJJn,https://api.spotify.com/v1/tracks/26xfrX9phl1N...,https://api.spotify.com/v1/audio-analysis/26xf...,156530,4,0.0
410,Give Yourself A Try,4rmIfFUZhhi9sS5IYtpkXw,The 1975,3mIj9lX2MWuHmhNCA7LSCW,1,196.588,0.334,0.786,11,-5.002,...,0.5170,0.901,182.933,audio_features,spotify:track:4rmIfFUZhhi9sS5IYtpkXw,https://api.spotify.com/v1/tracks/4rmIfFUZhhi9...,https://api.spotify.com/v1/audio-analysis/4rmI...,196589,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3962,Weightless,4cqYUqmKMcb3q1tdImVpGW,All Time Low,46gyXjRIvN1NL1eCB8GBxo,1,198.000,0.497,0.923,2,-4.512,...,0.1580,0.667,181.075,audio_features,spotify:track:4cqYUqmKMcb3q1tdImVpGW,https://api.spotify.com/v1/tracks/4cqYUqmKMcb3...,https://api.spotify.com/v1/audio-analysis/4cqY...,198000,4,0.0
4009,I'm Not A Vampire,1ar5WuMMxBf1bEsU4XwuD0,Falling In Reverse,2CmaKO2zEGJ1NWpS1yfVGz,1,231.840,0.235,0.965,2,-2.882,...,0.0448,0.322,185.278,audio_features,spotify:track:1ar5WuMMxBf1bEsU4XwuD0,https://api.spotify.com/v1/tracks/1ar5WuMMxBf1...,https://api.spotify.com/v1/audio-analysis/1ar5...,231840,4,0.0
4044,"Dear Maria, Count Me In",3ZsexY07D4t4HRq7ogeSAS,All Time Low,46gyXjRIvN1NL1eCB8GBxo,1,182.826,0.465,0.896,2,-3.126,...,0.2060,0.569,181.014,audio_features,spotify:track:3ZsexY07D4t4HRq7ogeSAS,https://api.spotify.com/v1/tracks/3ZsexY07D4t4...,https://api.spotify.com/v1/audio-analysis/3Zse...,182827,4,0.0
4139,Weightless,0lOm22W5svjv0JapB8EQ53,All Time Low,46gyXjRIvN1NL1eCB8GBxo,1,198.000,0.497,0.923,2,-4.512,...,0.1580,0.667,181.075,audio_features,spotify:track:0lOm22W5svjv0JapB8EQ53,https://api.spotify.com/v1/tracks/0lOm22W5svjv...,https://api.spotify.com/v1/audio-analysis/0lOm...,198000,4,0.0


In [68]:
# Make a function that will make a plus delta for tempos

def LowHigh(target_tempo, range):
    each_side = range/2
    lowtempo = target_tempo - each_side
    hightempo = target_tempo + each_side
    return lowtempo,hightempo

In [84]:
# Function to select songs within a range of tempos

def TempoRange(data,target_tempo,range):
    each_side = range/2
    lowtempo = target_tempo - each_side
    hightempo = target_tempo + each_side
    SongsInRange = data[(data['tempo'] > lowtempo) & (data['tempo'] < hightempo)]
    TrackIDRange = SongsInRange['Track_ID']
    return SongsInRange, TrackIDRange


In [85]:
(a,b) = TempoRange(data,target_tempo = 160,range =2)


In [44]:
# Now for playlists
# We need to create a playlist, then add the songs
# The function should take a string for the name
# The function should also break it up into batches if more than 100 songs

# user_playlist_create(user_id,name,public=True or False, collaborative = True or False)
# user_playlist_add_tracks(user,playlist_id,tracks)

scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

profile = sp.current_user()

user_id = profile['id']
p = sp.user_playlist_create(user_id,'TEST',public=False,collaborative=False)
playlist_id = p['id']

if len(b) >100 :
    # First get how many times 100 goes into the length
    times = math.ceil(len(b)/100)
    firstval = 0
    secondval = 100
    i =0
    while i <= times:
        if secondval < len(b):
            sp.user_playlist_add_tracks(user_id,playlist_id,tracks = b[firstval:secondval])
            firstval = secondval 
            secondval = secondval + 100
            i = i+1
            
        else:
            secondval = len(b)      
            sp.user_playlist_add_tracks(user_id,playlist_id,tracks = b[firstval:secondval])
            break
        

#sp.user_playlist_add_tracks(user_id,playlist_id,tracks = b[0:100])


In [89]:
# Make a function to do the playlists
def PlaylistMaker(Tracks):
    import math
    # Get user_id
    scope = "user-library-read playlist-read-private playlist-modify-public playlist-modify-private"
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
    profile = sp.current_user()
    user_id = profile['id']
    # Get playlist_name
    playlist_name = input("Please enter playlist name: ")
    # Public or not
    Public = input("Would you like this playlist to be public or not? Note the response must be True or False: ")
    # Collaborative or not
    Collab = False
    #Collab = input("Would you like this playlist ot be collaborative or not? Note the response must be True or False: ")
    # Tracks needs to be an input vector

    # Create playlist
    p = sp.user_playlist_create(user_id, playlist_name, public=Public, collaborative = Collab)
    # Get playlist id
    playlist_id = p['id']

    # loop to add songs - note that only 100 songs can be added at a time
    if len(Tracks) < 100:
        sp.user_playlist_add_tracks(user_id,playlist_id,tracks = Tracks, position = None)
        print("Playlist Made")
    elif len(Tracks) > 100:
        # First get how many times 100 goes into the length, and round up to decide the number of loops
        timesin = math.ceil(len(Tracks)/100)
        # the loop will always start with the first 100 if the loop has more than 100, or the total number of tracks if less than
        firstval = 0
        secondval = 100
        i = 0
        while i <= timesin:
            if secondval < len(Tracks):
                sp.user_playlist_add_tracks(user_id, playlist_id, tracks = Tracks[firstval:secondval],position = None)
                firstval = secondval
                secondval = secondval + 100 # move to the next 100
                i = i + 1 # next loop iteration
            else: 
                secondval = len(Tracks) # if you have less than 100 in a group of tracks
                sp.user_playlist_add_tracks(user_id, playlist_id, tracks = Tracks[firstval:secondval], position = None)
                print("Playlist Made")
                
    

In [90]:
len(b)

78

In [91]:
PlaylistMaker(b)

Playlist Made


In [64]:
LowHigh(150,5)

(147.5, 152.5)

In [79]:
math.ceil(len(b)/100)

1

In [82]:
a

,Track_Name,Track_ID,Artist_Name,Artist_ID,Artist_Num,Track_Len,danceability,energy,key,loudness,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,index
30,Bluffin (feat. Lil Baby),1hS8xbWKEifaf0VuFpIY69,Gucci Mane,13y7CgLHjMVRMDqxdx0Xdo,2,148.302,0.678,0.483,1,-7.956,...,0.1090,0.120,160.156,audio_features,spotify:track:1hS8xbWKEifaf0VuFpIY69,https://api.spotify.com/v1/tracks/1hS8xbWKEifa...,https://api.spotify.com/v1/audio-analysis/1hS8...,148302,4,0.0
31,Caro,6it15CsDlkqB7N4lF0C1qM,Bad Bunny,4q3ewBCX7sLwd24euuV69X,1,229.455,0.651,0.575,0,-7.739,...,0.0961,0.248,160.007,audio_features,spotify:track:6it15CsDlkqB7N4lF0C1qM,https://api.spotify.com/v1/tracks/6it15CsDlkqB...,https://api.spotify.com/v1/audio-analysis/6it1...,229456,4,0.0
103,Spoil My Night (feat. Swae Lee),5VuxWXbt7XENQCtE9TzpTv,Post Malone,246dkjvS1zLTtiykXe5h60,2,194.560,0.672,0.717,1,-2.714,...,0.1060,0.225,160.981,audio_features,spotify:track:5VuxWXbt7XENQCtE9TzpTv,https://api.spotify.com/v1/tracks/5VuxWXbt7XEN...,https://api.spotify.com/v1/audio-analysis/5Vux...,194560,4,0.0
138,yup!,4OUY3FgUqy3UAqDQHnsaa6,Yung Gravy,2YOYua8FpudSEiB9s88IgQ,1,189.000,0.769,0.744,9,-4.934,...,0.0784,0.592,160.118,audio_features,spotify:track:4OUY3FgUqy3UAqDQHnsaa6,https://api.spotify.com/v1/tracks/4OUY3FgUqy3U...,https://api.spotify.com/v1/audio-analysis/4OUY...,189000,4,0.0
228,Lego House,5ubHAQtKuFfiG4FXfLP804,Ed Sheeran,6eUKZXaKkcviH0Ku9w2n3V,1,185.093,0.592,0.637,11,-8.480,...,0.1300,0.565,159.701,audio_features,spotify:track:5ubHAQtKuFfiG4FXfLP804,https://api.spotify.com/v1/tracks/5ubHAQtKuFfi...,https://api.spotify.com/v1/audio-analysis/5ubH...,185093,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4138,Secret Valentine,4EzLeGdyuqoc0OvhkxtNMI,We The Kings,3ao3jf5d70Tf4fPh2bnXVl,1,206.880,0.397,0.921,1,-3.971,...,0.2000,0.317,159.980,audio_features,spotify:track:4EzLeGdyuqoc0OvhkxtNMI,https://api.spotify.com/v1/tracks/4EzLeGdyuqoc...,https://api.spotify.com/v1/audio-analysis/4EzL...,206880,4,0.0
4158,Headlines Read Out...,1oaafSJdIZs00dAcm14IgU,We The Kings,3ao3jf5d70Tf4fPh2bnXVl,1,192.506,0.467,0.932,1,-2.728,...,0.1230,0.704,159.925,audio_features,spotify:track:1oaafSJdIZs00dAcm14IgU,https://api.spotify.com/v1/tracks/1oaafSJdIZs0...,https://api.spotify.com/v1/audio-analysis/1oaa...,192507,4,0.0
4203,The Reckless and the Brave,4tfgqvaQinzshcm2JxU9Ap,All Time Low,46gyXjRIvN1NL1eCB8GBxo,1,199.491,0.459,0.993,1,-2.365,...,0.2220,0.642,160.114,audio_features,spotify:track:4tfgqvaQinzshcm2JxU9Ap,https://api.spotify.com/v1/tracks/4tfgqvaQinzs...,https://api.spotify.com/v1/audio-analysis/4tfg...,199491,4,0.0
4272,Duality,1ko1hqVxyzvRlAsbklLIbV,Fractal,5XkWlDmNLDZA8MiOaRm6HQ,1,360.000,0.482,0.904,5,-5.944,...,0.3990,0.397,159.950,audio_features,spotify:track:1ko1hqVxyzvRlAsbklLIbV,https://api.spotify.com/v1/tracks/1ko1hqVxyzvR...,https://api.spotify.com/v1/audio-analysis/1ko1...,360000,4,0.0
